<a href="https://colab.research.google.com/github/minit88/ML-DL/blob/main/Lab_09/01_mnist_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torchvision.datasets as dsets # 이미지 전처리
import torchvision.transforms as transforms # 이미지 전처리
import random

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777) # 기본적인 torch.rand() , torch.randint()와 같은 함수들에 대해서 seed적용
if device =='cuda':
  torch.cuda.manual_seed_all(777) # 현재 GPU를 통해 만들어내는 결과들에 랜덤성을 통제

In [12]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [13]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
# 첫번째 인자root 는 MNIST데이터를 다운 받을 경로 두번째 인자는 train은 인자로 트루를 주면 MNIST훈련 데이터를 리턴받음 FALSE 테스트 데이터 리턴
# 세번쨰 인자 transform은 현재 데이터를파이토치 텐서로 변환
# 네번째 인자 download는 해당 경로에 MNIST 데이터가 없다면 다운로드 받겠다는 의미

In [14]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True,drop_last=True)
# 첫번째 인자는 로드할 대상
# 두번쨰 인자는 배치 크기
# 세번째 인자는 에폭마다 미니배치를 셔플
# 네번째 인자는 마지막 배치를 버릴 것인지

In [15]:
linear = torch.nn.Linear(784,10,bias=True).to(device)
# to 함수는 연산을 어디서 수행할지를 정함

In [16]:
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.1953, -0.4404,  0.0139,  ...,  0.7510,  1.5190,  0.9637],
        [ 0.4146, -1.0660, -0.9969,  ..., -0.3259, -0.2695, -1.3316],
        [ 0.2600, -1.1821,  0.0065,  ...,  1.0648, -1.2251,  0.9841],
        ...,
        [ 0.0279, -1.7242, -1.1723,  ...,  1.3651, -0.0689, -0.7931],
        [ 0.0952,  0.1939,  0.1636,  ..., -1.0489,  0.6224, -1.7181],
        [ 2.6504,  0.6297, -1.6571,  ..., -0.0781, -1.1398, -0.8711]],
       requires_grad=True)

In [17]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(linear.parameters(),lr=learning_rate)

In [18]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader:
    X=X.view(-1,28*28).to(device)
    Y=Y.to(device)

    hypothesis = linear(X)
    cost = criterion(hypothesis,Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost +=cost/total_batch
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 5.728240490
Epoch: 0002 cost = 1.562848449
Epoch: 0003 cost = 1.036820769
Epoch: 0004 cost = 0.828017235
Epoch: 0005 cost = 0.710269213
Epoch: 0006 cost = 0.633605897
Epoch: 0007 cost = 0.577520132
Epoch: 0008 cost = 0.535485625
Epoch: 0009 cost = 0.502551138
Epoch: 0010 cost = 0.476104587
Epoch: 0011 cost = 0.453757554
Epoch: 0012 cost = 0.435718447
Epoch: 0013 cost = 0.420195490
Epoch: 0014 cost = 0.406820923
Epoch: 0015 cost = 0.394985646
Learning finished


In [29]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.891700029373169
Label:  5
Prediction:  5
